In [1]:
%load_ext autoreload
%autoreload 2


In [4]:
# Yuck its such a mess to get these libraries working... right now I install both with pip and colcon... that seems to do the trick. I install them both in editable way... It should handle the overlay ok.

from rgb_matters.data.utils.label_loader import LabelLoader
from rgb_matters.data.utils.vis_label import vis_grasp, vis_grasp_view_angle
from rgb_matters.constant import GRASPNET_ROOT, LABEL_DIR

from graspnetAPI.utils.eval_utils import transform_points, create_table_points
from graspnetAPI.utils.utils import generate_scene_model

from graspnetAPI import GraspNet, GraspNetEval
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import cv2
import yaml
from bam_utils.heatmap_helper import show_heatmap_img, show_heatmap_img_grid

In [7]:
config_path = "/home/bam/public_ws/src/rgb_matters/config/rs_rs_no_norm.yaml"

with open(config_path, "r") as f:
    train_config = yaml.load(f, Loader=yaml.FullLoader)

print(train_config)

batch_size = train_config["batch_size"]
eval_batch_size = train_config["eval_batch_size"]
eval_test_batch_num = train_config["eval_train_batch_num"]
eval_train_batch_num = train_config["eval_train_batch_num"]
train_camera = train_config["train_camera"]
test_camera = train_config["test_camera"]
num_layers = train_config["num_layers"]
lr = float(train_config["lr"])
iters = train_config["iters"]
use_normal = train_config["use_normal"]
normal_only = train_config["normal_only"]
lr_decay = train_config["lr_decay"]
grad_clip = train_config["grad_clip"]

grayscale = train_config["augmentation"]["grayscale"]
colorjitter_scale = train_config["augmentation"]["colorjitter_scale"]
random_crop = train_config["augmentation"]["random_crop"]
pos_weight = train_config["loss"]["pos"]
neg_weight = train_config["loss"]["neg"]
test_split = train_config["split"]["test_split"]
train_split = train_config["split"]["train_split"]

{'batch_size': 8, 'eval_batch_size': 1, 'eval_train_batch_num': 80, 'eval_test_batch_num': 80, 'train_camera': 'realsense', 'test_camera': 'realsense', 'lr': '4e-4', 'lr_decay': 0.8, 'grad_clip': 5.0, 'iters': 120000, 'num_layers': 50, 'use_normal': False, 'normal_only': False, 'augmentation': {'grayscale': False, 'colorjitter_scale': 0, 'random_crop': 0}, 'split': {'test_split': 'test', 'train_split': 'train'}, 'loss': {'pos': 1.0, 'neg': 1.0}}


In [ ]:
train_label_root = LABEL_DIR
test_label_root = LABEL_DIR


net = RGBNormalNet(
    num_layers=num_layers, use_normal=use_normal, normal_only=normal_only
)
net.train()